In [10]:
import os
import sys
import time
import shutil
import json
import random
import traceback
import numpy as np
import pandas as pd
from pathlib import Path
from statistics import mean
import torch
import pyvista as pv
import sys
import os
import json
import pymskt.mesh.io as io
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy
from NSM.models import TriplanarDecoder
from NSM.mesh import create_mesh
from pymskt.mesh.meshTools import get_cartilage_properties_at_points
from pymskt.mesh import Mesh, BoneMesh, CartilageMesh
from NSM.mesh.interpolate import interpolate_points, interpolate_mesh, interpolate_common
from pymskt import mesh

In [17]:
import os
import sys
import time
import shutil
import json
import random
import traceback
import numpy as np
import pandas as pd
from pathlib import Path
from statistics import mean
import torch
import pyvista as pv
import vtk
from vtk.util.numpy_support import numpy_to_vtk, vtk_to_numpy
from NSM.models import TriplanarDecoder
from NSM.mesh import create_mesh
from pymskt.mesh.meshTools import get_cartilage_properties_at_points
from pymskt.mesh import Mesh, BoneMesh, CartilageMesh
from NSM.mesh.interpolate import interpolate_points, interpolate_mesh, interpolate_common
from pymskt import mesh

    
def interpolate_matching_points(ref_latent, ref_mesh_path_femur, target_latent, model, model_type):
    if model_type == 'bone_only':
        ref_mesh = BoneMesh(ref_mesh_path_femur[0])
        ref_mesh_points= ref_mesh.points
        mesh_points_interp = interpolate_points(
            model= model,
            latent1= ref_latent,
            latent2= target_latent,
            n_steps= 100,
            points1= ref_mesh_points,
            surface_idx= 0,
            verbose= False,
            spherical= False
        )
        
        # interpolated_mesh= interpolate_mesh(
        #     model= model,
        #     latent1= ref_latent,
        #     latent2= target_latent,
        #     n_steps= 100,
        #     mesh= ref_mesh, 
        #     surface_idx= 0, 
        #     verbose= False, 
        #     spherical= True, 
        #     max_edge_len= 0.04, 
        #     adaptive= True, 
        #     smooth= True, 
        #     smooth_type= 'laplacian')
        
        # interpolated_mesh.resample_surface(clusters=10000)
        
        # create copies of the meshes, update the points, and save them to disk
        interpolated_mesh = ref_mesh.copy()
        interpolated_mesh.point_coords = mesh_points_interp
        return interpolated_mesh
        
    elif model_type == 'bone_cartilage':
        ref_mesh = BoneMesh(ref_mesh_path_femur[0])
        ref_mesh_points= ref_mesh.points
        mesh_points_interp_femur= interpolate_points(
            model=model,
            latent1= ref_latent,
            latent2= target_latent,
            n_steps= 100,
            points1= ref_mesh_points,
            surface_idx= 0,
            verbose= False,
            spherical= False
        )
        
        # interpolated_mesh_femur= interpolate_mesh(
        #     model= model,
        #     latent1= ref_latent,
        #     latent2= target_latent,
        #     n_steps= 100,
        #     mesh= ref_mesh, 
        #     surface_idx= 0, 
        #     verbose= False, 
        #     spherical= True, 
        #     max_edge_len= 0.04, 
        #     adaptive= True, 
        #     smooth= True, 
        #     smooth_type= 'laplacian')
        
        # interpolated_mesh_femur.resample_surface(clusters=10000)
        
        # interpolated_mesh_femur.copy_scalars_from_other_mesh_to_current(ref_mesh)
        
        # # create copies of the meshes, update the points, and save them to disk
        interpolated_mesh_femur= ref_mesh.copy()
        interpolated_mesh_femur.point_coords = mesh_points_interp_femur
        
        ref_mesh_cart = CartilageMesh(ref_mesh_path_femur[1])
        ref_mesh_points_cart= ref_mesh_cart.points
        mesh_points_interp_cart= interpolate_points(
            model=model,
            latent1= ref_latent,
            latent2= target_latent,
            n_steps= 100,
            points1= ref_mesh_points_cart,
            surface_idx= 1,
            verbose= False,
            spherical= False
        )
        
        # interpolated_mesh_cart= interpolate_mesh(
        #     model= model,
        #     latent1= ref_latent,
        #     latent2= target_latent,
        #     n_steps= 100,
        #     mesh= ref_mesh_cart, 
        #     surface_idx= 1, 
        #     verbose= False, 
        #     spherical= True, 
        #     max_edge_len= 0.04, 
        #     adaptive= True, 
        #     smooth= True, 
        #     smooth_type= 'laplacian')
        
        # interpolated_mesh_cart.resample_surface(clusters=10000)    
            
        # # create copies of the meshes, update the points, and save them to disk
        interpolated_mesh_cart= ref_mesh_cart.copy()
        interpolated_mesh_cart.point_coords = mesh_points_interp_cart
        
        return interpolated_mesh_femur, interpolated_mesh_cart
    else:
        raise ValueError(f'Unknown model type: {model_type}')

def compute_signed_distance_and_normal(mesh1_path, mesh2_path):
    """
    Compute signed distance from each point in mesh2 to the surface of mesh1.
    Signed distance is Positive outside, negative inside, zero on surface.
    Compute normal vector from mesh1 to mesh2.
    """
    mesh1 = pv.read(mesh1_path)
    mesh2 = pv.read(mesh2_path)
    
    assert mesh1.n_points == mesh2.n_points, "Resampling failed to produce matching number of points"

    # Build implicit distance function from surface of mesh1
    implicit_distance = vtk.vtkImplicitPolyDataDistance()
    surf = mesh1.extract_surface().triangulate().clean()
    implicit_distance.SetInput(surf)  # For older VTK versions

    # Evaluate signed distance at each point in mesh2
    signed_distances = np.array([
        implicit_distance.EvaluateFunction(point) for point in mesh2.points
    ])

    # Store result as point scalar
    mesh2.point_data["signed_distance"] = signed_distances
        
    # # Compute displacement vectors
    # disp_vectors = mesh2.points - mesh1.points  # shape (N, 3)
    # normx = disp_vectors[:, 0]
    # normy = disp_vectors[:, 1]
    # normz = disp_vectors[:, 2]
    # magnitude = np.linalg.norm(disp_vectors, axis=1)

    # # Attach as scalars to mesh2
    # mesh2.point_data['normx'] = normx
    # mesh2.point_data['normy'] = normy
    # mesh2.point_data['normz'] = normz
    
    # displacement_vectors = np.column_stack([
    # mesh2.point_data['normx'],
    # mesh2.point_data['normy'],
    # mesh2.point_data['normz']
    # ])
    # mesh2.point_data['displacement'] = displacement_vectors  # single vector field
    mesh = BoneMesh(mesh2)
    return mesh 

def get_icp_transform(model_path, bscore):
    
    '''
    This function generates mean mesh for a given Bscore along the Bscore vector from Mean healthy --> Mean OA
    '''
    
    import numpy as np
    import os
    import json

    path_json = os.path.join(model_path, 'model.json')

    with open(path_json, 'r') as f:
        model = json.load(f)
        
        bscore_vector = np.array(model['bscore_vector'])
        mean_healthy = np.array(model['mean_healthy'])
        std_healthy = np.array(model['std_healthy'])
        
        projection = (bscore * mean_healthy ) / std_healthy

        latent_vector = projection * bscore_vector
        
        return latent_vector

def interpolate_latent_for_bscore(model_path, bscore):
    """
    Interpolate latent vector for a given Bscore.
    """
    path_json = os.path.join(model_path, 'model.json')

    with open(path_json, 'r') as f:
        model = json.load(f)
        
        bscore_vector = np.array(model['bscore_vector'])
        mean_healthy = np.array(model['mean_healthy'])
        std_healthy = np.array(model['std_healthy'])
        
        # Compute norm squared of the bscore_vector
        bscore_vector_norm = np.linalg.norm(bscore_vector)

        # Your existing code
        projection = bscore * std_healthy + mean_healthy
        latent_vector = (projection / bscore_vector_norm) * bscore_vector
        
        # latent_vector = projection * bscore_vector
        
        return latent_vector

def project_fc_thickness_on_femur_mesh(bone_mesh_path, cart_mesh_path):

    bone_mesh= BoneMesh(bone_mesh_path)
    cart_mesh = CartilageMesh(cart_mesh_path)

    # IMPORTANT: The meshes should be "unnormailised" o patient size. Rough factor for scale-up is 45
    bone_mesh.point_coords = bone_mesh.point_coords * 45
    cart_mesh.point_coords = cart_mesh.point_coords * 45

    # setup the probe that we are using to get data from the T2 file 
    line_resolution = 100   # number of points along the line that the T2 data is sampled at
    no_thickness_filler = 0.0  # if no data is found, what value to fill the data with
    ray_cast_length= 10          # how far to extend the ray from the surface (using negative to go inwards/towards the other side)
    percent_ray_length_opposite_direction = 0.05  # extend the other way a % of the line to make sure get both edges. 1.0 = 100%|
    n_intersections = 2  # how many intersections to expect. If 2, then we expect to find two points on the ray that intersect the cartilage surface.

    thicknesses = np.zeros(bone_mesh.GetNumberOfPoints())

    node_data = get_cartilage_properties_at_points(
        bone_mesh,
        cart_mesh,
        t2_vtk_image=None,
        seg_vtk_image=None,
        ray_cast_length=ray_cast_length,
        percent_ray_length_opposite_direction=percent_ray_length_opposite_direction,
    )
    thicknesses += node_data

    # Assign the thickness scalars to the bone mesh surface.
    bone_mesh.point_data["thickness (mm)"] = thicknesses
    bone_mesh.point_coords = bone_mesh.point_coords / 45
    cart_mesh.point_coords = cart_mesh.point_coords / 45
    bone_mesh.save_mesh(bone_mesh_path)
    cart_mesh.save_mesh(cart_mesh_path)  
   
def calculate_fc_thickess_changes(bone_mesh_path_ref, bone_mesh_path_target):
    
    """
    Calculate the difference in cartilage thickness between two bone meshes.
    This function copies the cartilage thickness scalars from the reference mesh to the target mesh,
    calculates the difference in thickness, and saves the modified target mesh with the new scalars.
    Args:
        bone_mesh_path_ref (str): Path to the reference bone mesh file.
        bone_mesh_path_target (str): Path to the target bone mesh file.
    Returns:
        None: The function modifies the target mesh in place and saves it with the new scalars.
    
    """
    # Load the meshes
    bone_mesh1 = BoneMesh(bone_mesh_path_ref) 
    bone_mesh2 = BoneMesh(bone_mesh_path_target)
    
    bone_mesh2.copy_scalars_from_other_mesh_to_current(
        other_mesh=bone_mesh1,  # The mesh to copy scalars from
        orig_scalars_name='thickness (mm)',
        new_scalars_name= 'thickness (mm) Ref'
    )
    
    thickness_ref = vtk_to_numpy(bone_mesh2.GetPointData().GetArray('thickness (mm) Ref'))
    thickness = vtk_to_numpy(bone_mesh2.GetPointData().GetArray('thickness (mm)'))
    
    # Add edge detection - points with very low thickness are likely edge points
    edge_threshold = 0.25  # mm - adjust this value based on your data
    potential_edges = (thickness < edge_threshold) | (thickness_ref < edge_threshold)
    
    # Calculate the difference in thickness
    thickness_diff = thickness - thickness_ref
    
    # Set the difference to zero for edge points and where either thickness is zero
    thickness_diff[(thickness_ref == 0) | (thickness == 0) | potential_edges] = 0
    # thickness_diff[(thickness_ref == 0) | (thickness == 0)] = 0
    
    # Optional: Apply smoothing near edges to reduce artifacts
    # Find points near edges (within 2 points connectivity)
    near_edges = np.zeros_like(potential_edges)
    mesh_points = bone_mesh2.GetPoints()
    locator = vtk.vtkPointLocator()
    locator.SetDataSet(bone_mesh2)
    locator.BuildLocator()
    
    for i in range(len(potential_edges)):
        if potential_edges[i]:
            point = mesh_points.GetPoint(i)
            # Find points within a small radius
            id_list = vtk.vtkIdList()
            locator.FindPointsWithinRadius(0.5, point, id_list)  # 0.5mm radius
            for j in range(id_list.GetNumberOfIds()):
                near_edges[id_list.GetId(j)] = 1
    
    # Smooth the transition near edges
    for i in range(len(thickness_diff)):
        if near_edges[i] and not potential_edges[i]:
            # Find neighboring points
            id_list = vtk.vtkIdList()
            locator.FindClosestNPoints(5, mesh_points.GetPoint(i), id_list)
            # Calculate weighted average excluding edge points
            valid_diffs = []
            for j in range(id_list.GetNumberOfIds()):
                idx = id_list.GetId(j)
                if not potential_edges[idx]:
                    valid_diffs.append(thickness_diff[idx])
            if valid_diffs:
                thickness_diff[i] = np.mean(valid_diffs)
    
    # Add the difference scalar to the mesh
    bone_mesh2['thickness_diff (mm)'] = thickness_diff
    
    # Save the modified mesh
    bone_mesh2.save_mesh(bone_mesh_path_target)

In [20]:
aclr_path1 = '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model_trial/shape_change_overtime_raw/aclr/MeanFemur_NSMrecon_aclr_Visit-1.vtk'
aclr_path2= 'DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model_trial/shape_change_overtime_raw/aclr/MeanCart_NSMrecon_aclr_Visit-1_to_Visit-2.vtk'
aclr_cart_path1= '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model_trial/shape_change_overtime_raw/aclr/MeanCart_NSMrecon_aclr_Visit-1.vtk'
save_path = '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model_trial/shape_change_overtime_raw/'

In [19]:
mesh_org = pv.read(aclr_path1)

# Calculate the centroid of the mesh
centroid = np.mean(mesh_org.points, axis=0)

# Translate the mesh to center it at the origin (0, 0, 0)
mesh = mesh_org.translate(-centroid)  # Move to origin

# Get the bounds of the mesh
bounds = mesh.bounds

# Calculate the dimensions in each axis
dimension_x = bounds[1] - bounds[0]
dimension_y = bounds[3] - bounds[2]
dimension_z = bounds[5] - bounds[4]

print(f"Dimensions: x={dimension_x}, y={dimension_y}, z={dimension_z}")

if dimension_z > 0.7 * dimension_y: # if the S-I dimension is greater than 70% of the R-L dimension
    # Define the cropping parameters
    z_min = bounds[4]   
    z_max = bounds[4] + 0.7 * abs(bounds[1] - bounds[0])  # Define your maximum x value

    # Debugging printed values
    # print(f"Org z bounds: z_min={bounds[4]}, z_max={bounds[5]}")
    # print(f"New z bounds: z_min= {z_min}, z_max={z_max}")

    # Clip along z-axis
    clipped_mesh = mesh.clip("z", value=z_max, invert=True) 

    clip_bounds= clipped_mesh.bounds
    # print(f"Clip z bounds: z_min={clip_bounds[4]}, z_max={clip_bounds[5]}")
    # print(f"Num of points before clipping: {mesh.n_points}")
    # print(f"Num of points remaining after clipping: {clipped_mesh.n_points}")

else: # if the S-I dimension is less than 70% then just crop to 90% of the current S-I height to open up the top surface
    # Define the cropping parameters
    z_min = bounds[4]   
    z_max = bounds[4] + 0.95 * abs(bounds[5] - bounds[4])  # Define your maximum x value

    # Debugging printed values
    print(f"Org z b/s: z_min= {z_min}, z_max={z_max}")

    # Clip along z-axis
    clipped_mesh = mesh.clip("z", value=z_max, invert=True) 

    clip_bounds= clipped_mesh.bounds
    # print(f"Clip z bounds: z_min={clip_bounds[4]}, z_max={clip_bounds[5]}")
    # print(f"Num of points before clipping: {mesh.n_points}")
    # print(f"Num of points remaining after clipping: {clipped_mesh.n_points}")

# Translate the mesh back to its original position
clipped_mesh = clipped_mesh.translate(centroid)

bone_name= 'femur'
file_path= '/dataNAS/people/anoopai/DESS_ACL_study/results/BScore_and_FC_metrics/mean_shape_recon/BC_model_trial/shape_change_overtime_raw/aclr/MeanFemur_NSMrecon_aclr_Visit-1_clipped.vtk'
io.write_vtk(clipped_mesh, file_path, points_dtype=float, write_binary=False)

Dimensions: x=1.35460512064, y=1.07283564901, z=1.1287763907000001


In [ ]:
model_type = 'bone_cartilage'  # or 'bone_only'
model 
mean_latent_save_path = os.path.join(save_path, f'mean_latent_{knee}.npz')
latent= np.load(mean_latent_save_path) 

ref_latent = latent['Visit-1'].squeeze()
target_latent = latent["Visit-5"].squeeze()

interp_mesh_femur, interp_mesh_cart= interpolate_matching_points(ref_latent, [file_path, aclr_cart_path1], target_latent, model, model_type)
interp_mesh_path_femur =os.path.join(mesh_save_path, f'MeanFemur_NSMrecon_{knee}_Visit-1_to_{visit}.vtk')    
interp_mesh_path_cart =os.path.join(mesh_save_path, f'MeanCart_NSMrecon_{knee}_Visit-1_to_{visit}.vtk')

interp_mesh_femur.save_mesh(interp_mesh_path_femur)
interp_mesh_cart.save_mesh(interp_mesh_path_cart)  
project_fc_thickness_on_femur_mesh(interp_mesh_path_femur, interp_mesh_path_cart)
        
print(f'Computing deformation for {knee} knee from Visit-1 to {visit}')
ref_mesh_path_femur =os.path.join(mesh_save_path, f'MeanFemur_NSMrecon_{knee}_Visit-1.vtk')
ref_mesh_path_cart =os.path.join(mesh_save_path, f'MeanCart_NSMrecon_{knee}_Visit-1.vtk')
deformed_mesh_femur = compute_signed_distance_and_normal(ref_mesh_path_femur, interp_mesh_path_femur)
deformed_mesh_cart = compute_signed_distance_and_normal(ref_mesh_path_cart, interp_mesh_path_cart)
deformed_mesh_femur.save_mesh(interp_mesh_path_femur)
deformed_mesh_cart.save_mesh(interp_mesh_path_cart)
calculate_fc_thickess_changes(ref_mesh_path_femur, interp_mesh_path_femur)